# Modelo Predictivo

## 1.- lectura del CSV con los datos preprocesados de la estación de Aotizhongxin

In [243]:
import pandas as pd
from datetime import datetime
df = pd.read_csv(filepath_or_buffer='C:/Users/Edgar/Documents/GitHub/DataMining_and_MachineLearning_EdgarV/data/Proyecto2/calidad_aire_procesado.csv',sep=',')
## se pensaba colocar 3 columnas pero esto solo aumentaria la dimensionalidad
'''df[['Anio','Mes','Dia']] = df['fecha'].str.split('-', expand=True)'''
## se decidió transformar la fecha al número de dias medidos desde el primero de enero de 1970, en la aplicación transformar lo fecha en valor de numero de dias
df['fecha'] = pd.to_datetime(df['fecha'])
df['dias'] = (df['fecha'] - datetime(1970, 1, 1)).dt.days
## se definió un rango y una categoría a los valores de AQI con la intensión de aumentar la presición, el recall y el f1 score ya que al intentar entrenar cada valor necesita un volumen de datos mayor
rangos = [0, 50, 100, 150, 200, 250, 300, 10000]
categorias = ['Excelente', 'Bueno', 'poco contaminado', 'ligeramente contaminado','moderadamente contaminado','fuertemente contaminado','fuera del rango']
df['Categoria'] = pd.cut(df['AQI'], bins=rangos, labels=categorias)
df.head(50)

,fecha,AQI_SO2,AQI_NO2,AQI_PM10,AQI_CO,AQI_O3,AQI,dias,Categoria
0,2013-10-01,7,36,85,22,21,85,15979,Bueno
1,2013-10-10,26,27,92,17,21,92,15988,Bueno
2,2013-10-11,18,37,72,27,43,72,15989,Bueno
3,2013-10-12,3,41,66,13,37,66,15990,Bueno
4,2013-10-13,4,32,51,14,25,51,15991,Bueno
5,2013-10-14,2,24,35,11,17,35,15992,Excelente
6,2013-10-15,3,25,24,8,25,25,15993,Excelente
7,2013-10-16,4,34,64,13,29,64,15994,Bueno
8,2013-10-17,12,48,108,18,26,108,15995,poco contaminado
9,2013-10-18,22,65,150,30,11,150,15996,poco contaminado


## 2.- Ajustar los datos al modelo

In [244]:
## debido a que los valores mayores de 300 estan fuera del rango los descartamos para que no se produzca mayor ruido
categoria = 'fuera del rango'
cantidad = df['Categoria'].value_counts()[categoria]
cantidad
categoria_a_eliminar = 'fuera del rango'
df = df.query('Categoria != @categoria_a_eliminar')
cantidad

5

## 3.- Seleccionar los datos de entrada y de salida para ingresarlos al modelo

In [245]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
## dias es la variable para realizar la predicción, como era de esperarse la nueva columna categoría es la variable categórica de salida del modelo
X = df[['dias', 'AQI_SO2', 'AQI_NO2', 'AQI_PM10','AQI_CO','AQI_O3']]
y = df['Categoria']
## como SVM admite solamente valores numéricos lo que hacemos es etiquetar cada categoría con un número
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
## se ajustan los datos de entrenamiento y testeo en 70% y 30% respectivamente, elegimos la semilla numero 28 por su mayor valor de rendimiento
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=28)
print("listo")

listo


## 4.- Selección del modelo

In [251]:
from sklearn.svm import SVC

## Seleccioonamos el modelo SVM y ajustamos el kernel a linear debido a que hay mas de 2 dimensiones 
model = SVC(C=0.1,kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("listo")

listo


## 5.- Calcular valores de exactitud, presición, esfuerzo y F1

In [252]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
## haciendo todo lo posible haciendo un ajuste de datos con la desviación estandar y la media, descartando valores que generan ruido y categorizando se obtuvo los mejores valores hasta el momento

Accuracy: 0.819672131147541
Precision: 0.819672131147541
Recall: 0.819672131147541
F1 Score: 0.819672131147541


## 6.- Se evalua mediante cross validation

In [253]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5)
print("Cross-Validation Scores:", scores)
print("Average Accuracy:", scores.mean())

Cross-Validation Scores: [0.77192982 0.79649123 0.77464789 0.77816901 0.76056338]
Average Accuracy: 0.7763602668643439


## 7.- Se optimiza el aprendizaje del modelo tuneando los hiperparámetros.

In [254]:
from sklearn.model_selection import GridSearchCV


param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print('Best parameters', best_params)



Best parameters {'C': 0.1, 'kernel': 'linear'}


## 8.- Se realiza el Test

In [289]:
from datetime import datetime
def numero_dias(desde_fecha):
    fecha = datetime.strptime(desde_fecha, '%d/%m/%Y')
    fecha_base = datetime(1970, 1, 1)
    diferencia = fecha - fecha_base
    numero_dias = diferencia.days
    return numero_dias
## 2013-10-27 	19 	63 	136 	36 	6 	136 	16005 	poco contaminado
## 2013-10-28 	26 	61 	174 	57 	44 	174 	16006 	ligeramente contaminado

fecha = '28/10/2013'
valor1 = numero_dias(fecha)
valor2 = 26
valor3 = 61
valor4 = 174
valor5 = 57
valor6 = 44

dato_prediccion = [[valor1, valor2, valor3, valor4, valor5, valor6]]
prediccion = best_model.predict(dato_prediccion)


print("La categoría predicha para la fecha", fecha, "es:", prediccion)


La categoría predicha para la fecha 28/10/2013 es: [3]


C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
